In [2]:
# import useful libraries
import cv2
import matplotlib.pyplot as plt
import numpy as np
from statistics import mode
import tkinter as tk
from tkinter.filedialog import askopenfilenames 
from tkinter.filedialog import askdirectory

plt.rcParams['figure.figsize'] = [16, 10]

"""Filter out elements based on size
credit to: Professor Wolka(from demo Edge_Processing2)
"""
def conncomp_filter(img, mag_thresh, size_thresh):
    [w,h] = img.shape
    timg = np.zeros((w,h)) # initialize a new thresholded Laplacian image
    
    bimg = img > mag_thresh

    # Calculate the connected components; returns an image with a unique number assigned to each region (labels)
    (retval, labels, stats, centroids) = cv2.connectedComponentsWithStats(bimg.astype('uint8'))
    
    # Loop through the connected components to remove all with size below the size threshold
    for idx, stat in enumerate(stats):
        if idx > 0: # ignore the background element
            if stat[4] > size_thresh:
                timg[labels == idx] = 255

    return timg

def corp(img, path, filename, normalize):
    """input: img the location of the image you want to crop
    path the location of the place you want to save the cropped images
    filename what you want to name files
    normalize a boolean, true if you want normalize the images(in grayscale) false otherwise
    returns: writes the corpped images into the location path is
    """
    image = cv2.imread(img)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    (height,width) = gray.shape

    #finds keypoints that will be used to find the background color to create a mask
    keypoints = []
    #keypoints will be values along the four corners
    for i in range(1, min(height, width)):
        keypoints += [gray[0, i], gray[height - i, 0], gray[height - 1, i], gray[height -i, width - 1]]
    lower_gray = 0
    upper_gray = mode(keypoints) - 7

    #creates a mask
    thresh = cv2.threshold(gray, lower_gray, upper_gray, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

    if normalize:
        normalize_factor = 150 - upper_gray
        gray = gray + normalize_factor
        gray = np.array(gray, dtype=np.uint8)
        image = cv2.cvtColor(gray, cv2.COLOR_GRAY2RGB)

    #refines the mask by filtering out noise by filtering by size
    thresh = conncomp_filter(thresh, 10, 25000)
    thresh = thresh.astype(np.uint8)

    #variables to check if a corner is already cropped
    bottom_left_corner = False
    top_left_corner = False
    bottom_right_corner = False
    top_right_corner = False

    # Find contours, obtain bounding box, extract and save region of interest
    bounding_box_number = 0
    cnts = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]

    for c in cnts:
        x,y,w,h = cv2.boundingRect(c)

        #caculates the region of interest center coordinates
        cx = x+w//2
        cy = y+h//2
        #region is the size that is around the center coordinates
        region = 1000
        #if center coordinates results in a corp that is not within our
        #region size change the center coordinates
        if (cx < region):
            cx = region
        if (cy < region):
            cy = region
        if (height - cy < 1000):
            cy = height - 1000
        if (width - cx < 1000):
            cx = width - 1000
        
        bounding_box = image[cy-region:cy+region, cx-region:cx+region]

        #If a corner hasn't been cropped yet corp it
        if (cy == 1000 and cx == 1000 and not(bottom_left_corner)):
            cv2.imwrite(f'{path}/{filename}_{bounding_box_number}.png', bounding_box)
            bounding_box_number += 1
        elif (cy == height - 1000 and cx == 1000 and not(top_left_corner)):
            cv2.imwrite(f'{path}/{filename}_{bounding_box_number}.png', bounding_box)
            bounding_box_number += 1
        elif (cy == 1000 and cx == width - 1000 and not(bottom_right_corner)):
            cv2.imwrite(f'{path}/{filename}_{bounding_box_number}.png', bounding_box)
            bounding_box_number += 1
        elif (cy == height - 1000 and cx == width - 1000 and not(top_right_corner)):
            cv2.imwrite(f'{path}/{filename}_{bounding_box_number}.png', bounding_box)
            bounding_box_number += 1
        elif ((cy > 1000 and cx > 1000) or (cy < height - 1000 and cx < width - 1000)):
            cv2.imwrite(f'{path}/{filename}_{bounding_box_number}.png', bounding_box)
            bounding_box_number += 1

        #if a corner is corpped mark it as such
        if (cy == 1000 and cx == 1000):
            bottom_left_corner = True
        if (cy == height - 1000 and cx == 1000):
            top_left_corner = True
        if (cy == 1000 and cx == width - 1000):
            bottom_right_corner = True
        if (cy == height - 1000 and cx == width - 1000):
            top_right_corner = True


def files():
    """creates a simple GUI that assists with calling corp on multiple images,
    allows easily to choose multiple files to corp at once, name the files, 
    choose the location to save the corps, and if you want to normalize the 
    outputs
    Warning: images that you want to corp must have a name that is size greater than
    2(excluding the extension) as the last 2 charcters of the image name is used 
    as an additional unique idenfier of the new corps
    """
    #creates the root
    root = tk.Tk()
    root.geometry("400x150")
    root.title("Pollen Cropper")

    #create labels
    files = tk.Label(text ="Files to crop")
    files.grid(column=0, row=1)
    new_name = tk.Label(text = "Name of the new files")
    new_name.grid(column = 0, row = 2)
    place = tk.Label(text = "Place to save")
    place.grid(column = 0, row = 3)
    normal = tk.Label(text = "Normalize(greyscale)?")
    normal.grid(column = 0, row = 4)
    normalize = tk.BooleanVar()

    #functions that are called depending the button pressed
    def choosefiles():
        """saves the name of the files into a global variable filename
        when called upon
        """
        global filenames
        filenames = askopenfilenames(title = "Open 'png' or 'jpg' file")

    def chooselocation():
        """saves the location of the directory into a global variable location
        when called upon
        """
        global location
        location = askdirectory(title = "Open folder to save files to")
    
    def annotate():
        """loops through each file in filename and calls corp on each file when
        called upon
        """
        name = new_name_entry.get()
        for file in filenames:
            corp(file, location, name + file[-5:-3], normalize.get())

    #creates buttons
    button = tk.Button(root, text = "Choose Files", command = choosefiles, bg = "pink")
    button.grid(column = 1, row = 1)
    new_name_entry = tk.Entry()
    new_name_entry.grid(column = 1, row = 2)
    button = tk.Button(root, text = "Choose Location", command = chooselocation, bg = "pink")
    button.grid(column = 1, row = 3)
    check_box = tk.Checkbutton(root, text ="", variable = normalize, onvalue = True, offvalue = False)
    check_box.grid(column = 1, row = 4)
    button = tk.Button(root, text = "Crop", command = annotate, bg = "pink")
    button.grid(column = 1, row = 5)

    root.mainloop()

files()